<a href="https://colab.research.google.com/github/asmy101/challenge-big-data/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files 
from IPython.display import Image
uploa=files.upload()

In [ ]:
from google.colab import files 
from IPython.display import Image
uploa=files.upload()

In [ ]:
import pandas as pd
import csv

# Load the data into a pandas dataframe
df = pd.read_csv('comments_tamazight.csv')

# Show the first few rows of the dataframe
print(df.head())

In [ ]:
print(df['score'])

In [ ]:
pip install kafka.python

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
import csv 
import json 

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)
          
csvFilePath = r'comments_tamazight.csv'
jsonFilePath = r'comments.json'

csv_to_json(csvFilePath, jsonFilePath)

# Load the existing dataset into a pandas dataframe
df = pd.read_csv("comments_tamazight.csv")

# Randomly sample a portion of the data
df = df.sample(frac=0.9, random_state=1)

# Preprocess the text data
comments = df['comment'].tolist()
scores = df['score'].tolist()

# Tokenize the comments
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(comments)
sequence = tokenizer.texts_to_sequences(comments)
padded = pad_sequences(sequence, maxlen=100)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded, scores, test_size=0.2, random_state=1)
y_train = np.array(y_train)
y_val = np.array(y_val)


# Create the deep neural network model
model = Sequential([
    Embedding(10000, 32),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=32)

# Load the scraped data into a pandas dataframe
scraped_data = pd.read_json("comments.json")

# Extract the text field from the scraped data
scraped_comments = scraped_data['comment'].tolist()

# Preprocess the scraped comments
scraped_sequence = tokenizer.texts_to_sequences(scraped_comments)
scraped_padded = pad_sequences(scraped_sequence, maxlen=100)

# Make predictions on the scraped comments
scraped_scores = model.predict(scraped_padded)
print(scraped_scores)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier

# Load the existing dataset into a pandas dataframe
df = pd.read_csv("comments_tamazight.csv")

# Select a smaller subset of the data
subset = df.sample(frac=0.1)

# Preprocess the text data
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(subset['comment'])

# Train the model
model = DecisionTreeClassifier()
model.fit(features, subset['score'])

# Load the scraped data into a pandas dataframe
scraped_data = pd.read_json("comments.json")

# Extract the text field from the scraped data
scraped_comments = scraped_data['comment'].tolist()

# Preprocess the scraped comments
scraped_features = vectorizer.transform(scraped_comments)

# Make predictions on the scraped comments
scraped_scores = model.predict(scraped_features)
print(scraped_scores)

In [ ]:
# Evaluate the accuracy of the model
accuracy = model.score(features, subset['score'])
print("Accuracy: ", accuracy)

In [ ]:
# Create a new dataframe from the scraped comments and their scores
scraped_df = pd.DataFrame({'comments': scraped_comments, 'scores': scraped_scores})

# Iterate over the rows of the dataframe and print the comment and its score
for index, row in scraped_df.iterrows():
    print("Comment: ", row['comments'])
    print("Score: ", row['scores'])
    print("\n")

In [ ]:
print(scraped_df['scores'])

In [ ]:
#Create a sentiment column
sentiments = []
for score in scraped_scores:
    if score > 0:
        sentiments.append("positive")
    elif score < 0:
        sentiments.append("negative")
    else:
        sentiments.append("neutral")

#Add the sentiment column to the scraped_data dataframe
scraped_data['sentiment'] = sentiments

In [ ]:
print(scraped_data['sentiment'])

In [ ]:
print(scraped_data)

In [ ]:
#Create a new column in the scraped_data dataframe with the sentiment labels
scraped_data['sentiment'] = ['positive' if score > 0 else 'negative' if score < 0 else 'neutral' for score in scraped_scores]

#Save the updated scraped_data dataframe to a json file
scraped_data.to_json("scraped_data_with_sentiment.json")

In [ ]:
print(scraped_data)

In [ ]:
# Save the updated scraped_data to a json file
scraped_data.to_json("scraped_data_with_sentiment.json", orient='records', force_ascii=False)